In [27]:
import os
import csv
import json
import sys
import numpy as np
import random

import librosa

In [28]:
# sound_file = "../../databases/TESS_and_RAVDESS/Angry/03-01-05-01-01-01-01.wav"

# # sample rate is per second. 2 second video we have about 44,000 samples
# signal, sr = librosa.load(sound_file, sr=22050)

In [29]:
# mfcc = librosa.feature.mfcc(signal, n_fft=2048, hop_length=512, n_mfcc=13)

In [30]:
# li = [180, -720, 50, -600, -450, -50, -100]
# min_ = -720
# max_ = 180
# li = [2*((x-min_)/(max_-min_))-1 for x in li]
# print(li)

In [31]:
# print(mfcc)
# x = 2*((mfcc-min_)/(max_-min_))-1
# print(x)

In [32]:
sound_emotion_path = "../../databases/TESS_and_RAVDESS/"
train_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

validation_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

test_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

max_ = 0
min_ = 0

In [33]:
def process_sound(filename, dirpath, emotion, label, sample_rate, n_mfcc, n_fft, hop_length):
    file_path = os.path.join(dirpath, filename)
    # sample rate is per-second, signal.size/sample rate = time
    signal, sr = librosa.load(file_path, sr=sample_rate)
    # Option to round this so we get more samples, but having consistent 1 second samples is important too
    intervals = int(signal.size / sample_rate)
    for x in range(intervals):
        start = x * sample_rate
        stop = start + sample_rate
        mfcc = librosa.feature.mfcc(signal[start:stop], sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
        mfcc = mfcc.T

        randint = random.random()
        if randint < 0.8: 
            train_data["mfcc"].append(mfcc.tolist())
            train_data["label"].append(label)
            train_data["emotion"].append(emotion)
        elif randint >= 0.8 and randint < 0.9:
            validation_data["mfcc"].append(mfcc.tolist())
            validation_data["label"].append(label)
            validation_data["emotion"].append(emotion)            
        else:       
            test_data["mfcc"].append(mfcc.tolist())
            test_data["label"].append(label)
            test_data["emotion"].append(emotion)              


In [34]:
def save_mfcc(dataset_path, n_mfcc=13, n_fft=1024, hop_length=256, sample_rate=22050):
    # count: 0=Angry, 1=Disgusted, 2=Fearful, 3=Happy, 4=Netural, 5=Sad, 6=Surprised
    for count, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # print(filenames)
        # print(dirnames)
        print(dirpath)
        if dirpath is not sound_emotion_path:
            if "Angry" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "anger_disgust", 0, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Disgusted" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "anger_disgust", 0, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Suprised" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "surprised_fear", 4, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Fearful" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "surprised_fear", 4, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Happy" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "joy", 1, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Sad" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "sadness", 3, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Neutral" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "neutral", 2, sample_rate, n_mfcc, n_fft, hop_length)
            else:
                continue

save_mfcc(sound_emotion_path)

../../databases/TESS_and_RAVDESS/
../../databases/TESS_and_RAVDESS/Emotions
../../databases/TESS_and_RAVDESS/Emotions\Angry
../../databases/TESS_and_RAVDESS/Emotions\Disgusted
../../databases/TESS_and_RAVDESS/Emotions\Fearful
../../databases/TESS_and_RAVDESS/Emotions\Happy
../../databases/TESS_and_RAVDESS/Emotions\Neutral
../../databases/TESS_and_RAVDESS/Emotions\Sad
../../databases/TESS_and_RAVDESS/Emotions\Suprised


In [35]:
# probably need to create a new json object 
# for x in data["mfcc"]:
#     arr = np.asarray(x)
#     2*((arr-min_)/(max_-min_))-1
#     data2["mfcc"].append(arr.tolist())
# print(data["mfcc"])

In [36]:
with open("./tess_ravdess_train.json", "w") as fp:
    json.dump(train_data, fp, indent=4)
with open("./tess_ravdess_validation.json", "w") as fp:
    json.dump(validation_data, fp, indent=4)
with open("./tess_ravdess_test.json", "w") as fp:
    json.dump(test_data, fp, indent=4)